In [ ]:
'''
pip install ipythonblocks pillow bleak
'''

import time
import asyncio
import logging
import threading
from PIL import Image, ImageDraw, ImageFont, ImageSequence  # pillow
from IPython import display
from ipythonblocks import BlockGrid

from ledpanel import ledpanel

'''
async -> sync
    ref: https://github.com/intelino-code/intelino-trainlib-py/blob/master/intelino/trainlib/train.py
'''

# ref imagiCharms https://imagilabs.com/app
RED = R = (255, 0, 0)
GREEN = G = (0, 255, 0)
BLUE = B = (0, 0, 255)
ORANGE = O = (255, 165, 0)
YELLOW = Y = (255, 255, 0)
AQUA = A = (0, 255, 255)
PURPLE = P = (148, 0, 211)
MAGENTA = M = (255, 0, 255)
BLACK = K = (0, 0, 0)
WHITE = W = (255, 255, 255)

class LedBag:
    """Class LedBag(16x16) encapsulates the LedBag communication.
    import bluetooth;bluetooth.discover_devices(lookup_names=True) : get device id

    from codelab_adapter.led_bag import LedBag
    bag = LedBag()
    bag.conect()  # conect(device_id)
    bag.set_pixel(0, 0, 'red')
    """
    
    _COLOR = {
        "red": RED,
        "green": GREEN,
        "blue": BLUE,
        "orange": ORANGE,
        "yellow": YELLOW,
        "aqua": AQUA,
        "purple": PURPLE,
        "magenta": MAGENTA,
        "black": BLACK,
        "white": WHITE,
    }

    
    def __init__(self, host="local", port=1, logger=None):
        # async -> sync
        self.__event_loop = asyncio.new_event_loop()
        self.__lock = threading.Lock()
        self.__thread = threading.Thread(target=self.__event_loop.run_forever)
        self.__thread.start()
        
        self._ledpanel = None
        self.type = "LedBag"  # ...
        self.size = 16
        self.host = host
        self.port = 1
        self._img = Image.new(mode="RGB", size=(self.size, self.size))
        self._show_duration = 0.1
        self._show_scale = 20
        

        if logger is None:
            self.logger = logging.getLogger(self.type)
 
    def __del__(self):
        if self._ledpanel:
            self._execute(self._ledpanel.disconnect())
        
    def _execute(self, coroutine, timeout: float = None):
        with self.__lock:
            return asyncio.run_coroutine_threadsafe(
                coroutine, self.__event_loop
            ).result(timeout)
            
    def _is_local(self):
        if self.host == "local":
            return True
        else:
            return False

    def _imshow(self, img, PILimage=True):
        display.clear_output(wait=True)

        imdata = img.getdata()
        grid = BlockGrid(self.size, self.size, fill=(0, 0, 0), block_size=self._show_scale)
        for block, colors in zip(grid, imdata):
            block.rgb = colors
        display.display(grid)
        time.sleep(self._show_duration)
        
    def _show_image(self, image, PILimage=True):
        pass

    def _show_here(self, img=None, PILimage=True):
        if img:
            self._imshow(img, PILimage=PILimage)
        else:
            self._imshow(self._img, PILimage=PILimage)
            
    def connect(self, host=None):
        if host:
            self.host = host
        if self._is_local():
            return
        
        self._ledpanel = ledpanel(self.host)
        self._execute(self._ledpanel.connect())

    def set_pixel(self, x, y, color, show=True):
        '''设置 x, y 位置的颜色

        eg: set_pixel(0, 1) = (255, 0, 0)'''
        # https://microbit-micropython.readthedocs.io/en/v1.0.1/display.html#microbit.display.set_pixel
        if type(color) == str:
            color = self._COLOR[color]
        y, x = x, y  # x y 相反，与imagicharm保持一致
        self._img.putpixel((x, y), color)  # 左上角红色
        #if show:
        #    return self.show()
        if self._is_local():
            self.show()
        else:
            # to bag
            r, g, b = color
            self._execute(self._ledpanel.draw_point(r,g,b,x,y))
            # self._show_image(img, PILimage=PILimage)
        
    def show(self, img=None, PILimage=True):
        '''显示图像
        '''
        if not img:
            img = self._img
        if self._is_local():
            return self._show_here(img, PILimage=PILimage)
        else:
            # to bag
            self._show_image(img, PILimage=PILimage)

    def clear(self):
        self._execute(self._ledpanel.clear())

In [ ]:
addr = "ABE21030-5717-420D-BCC5-72467D554EC7"
bag = LedBag(addr)
bag.connect()

In [ ]:
import random
for i in range(16):
    for j in range(16):
        if i == j:
            bag.set_pixel(i, j, 'red')

In [ ]:
bag2 = LedBag()
bag2.connect()  #  默认连接 jupyterlab 模拟器，输入书包id可连接到书包: bag.connect(DEVICE_ID)

In [ ]:
for i in range(16):
    for j in range(16):
        if i == j:
            bag2.set_pixel(i, j, 'red')